## Udforsk K-Means clustering med R og principperne for Tidy data.

### [**Quiz før lektionen**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/29/)

I denne lektion vil du lære, hvordan man opretter klynger ved hjælp af Tidymodels-pakken og andre pakker i R-økosystemet (vi kalder dem venner 🧑‍🤝‍🧑) samt det nigerianske musikdatasæt, du importerede tidligere. Vi vil dække det grundlæggende i K-Means til clustering. Husk, som du lærte i den tidligere lektion, at der er mange måder at arbejde med klynger på, og metoden afhænger af dine data. Vi vil prøve K-Means, da det er den mest almindelige clustering-teknik. Lad os komme i gang!

Begreber, du vil lære om:

-   Silhouettescore

-   Elbow-metoden

-   Inerti

-   Varians

### **Introduktion**

[K-Means Clustering](https://wikipedia.org/wiki/K-means_clustering) er en metode, der stammer fra signalbehandling. Den bruges til at opdele og gruppere data i `k klynger` baseret på ligheder i deres egenskaber.

Klyngerne kan visualiseres som [Voronoi-diagrammer](https://wikipedia.org/wiki/Voronoi_diagram), som inkluderer et punkt (eller 'frø') og dets tilsvarende område.

<p >
   <img src="../../images/voronoi.png"
   width="500"/>
   <figcaption>Infografik af Jen Looper</figcaption>


K-Means clustering har følgende trin:

1.  Dataforskeren starter med at specificere det ønskede antal klynger, der skal oprettes.

2.  Derefter vælger algoritmen tilfældigt K observationer fra datasættet som de oprindelige centre for klyngerne (dvs. centroid).

3.  Herefter tildeles hver af de resterende observationer til det nærmeste centroid.

4.  Derefter beregnes de nye gennemsnit for hver klynge, og centroid flyttes til gennemsnittet.

5.  Når centrene er blevet genberegnet, kontrolleres hver observation igen for at se, om den måske er tættere på en anden klynge. Alle objekter tildeles igen ved hjælp af de opdaterede klyngegennemsnit. Trinnene med klyngetildeling og opdatering af centroid gentages iterativt, indtil klyngetildelingerne stopper med at ændre sig (dvs. når konvergens er opnået). Typisk afsluttes algoritmen, når hver ny iteration resulterer i ubetydelig bevægelse af centroids, og klyngerne bliver statiske.

<div>

> Bemærk, at på grund af tilfældigheden i valget af de oprindelige k observationer, der bruges som startcentroids, kan vi få lidt forskellige resultater hver gang vi anvender proceduren. Af denne grund bruger de fleste algoritmer flere *tilfældige starter* og vælger iterationen med den laveste WCSS. Derfor anbefales det stærkt altid at køre K-Means med flere værdier af *nstart* for at undgå et *uønsket lokalt optimum.*

</div>

Denne korte animation, der bruger [illustrationer](https://github.com/allisonhorst/stats-illustrations) af Allison Horst, forklarer clustering-processen:

<p >
   <img src="../../images/kmeans.gif"
   width="550"/>
   <figcaption>Illustration af @allison_horst</figcaption>



Et grundlæggende spørgsmål, der opstår i clustering, er dette: Hvordan ved du, hvor mange klynger dine data skal opdeles i? En ulempe ved at bruge K-Means er, at du skal fastlægge `k`, det vil sige antallet af `centroids`. Heldigvis hjælper `elbow-metoden` med at estimere en god startværdi for `k`. Du vil prøve det om lidt.

### 

**Forudsætning**

Vi fortsætter lige der, hvor vi slap i [den forrige lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb), hvor vi analyserede datasættet, lavede en masse visualiseringer og filtrerede datasættet til interessante observationer. Sørg for at tjekke det ud!

Vi skal bruge nogle pakker for at komme i gang med dette modul. Du kan installere dem som: `install.packages(c('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork'))`

Alternativt kan scriptet nedenfor tjekke, om du har de nødvendige pakker til at gennemføre dette modul, og installere dem for dig, hvis nogle mangler.


In [ ]:
suppressWarnings(if(!require("pacman")) install.packages("pacman"))

pacman::p_load('tidyverse', 'tidymodels', 'cluster', 'summarytools', 'plotly', 'paletteer', 'factoextra', 'patchwork')


Lad os komme i gang!

## 1. En dans med data: Indsnævr til de 3 mest populære musikgenrer

Dette er en opsummering af, hvad vi gjorde i den forrige lektion. Lad os analysere nogle data!


In [ ]:
# Load the core tidyverse and make it available in your current R session
library(tidyverse)

# Import the data into a tibble
df <- read_csv(file = "https://raw.githubusercontent.com/microsoft/ML-For-Beginners/main/5-Clustering/data/nigerian-songs.csv", show_col_types = FALSE)

# Narrow down to top 3 popular genres
nigerian_songs <- df %>% 
  # Concentrate on top 3 genres
  filter(artist_top_genre %in% c("afro dancehall", "afropop","nigerian pop")) %>% 
  # Remove unclassified observations
  filter(popularity != 0)



# Visualize popular genres using bar plots
theme_set(theme_light())
nigerian_songs %>%
  count(artist_top_genre) %>%
  ggplot(mapping = aes(x = artist_top_genre, y = n,
                       fill = artist_top_genre)) +
  geom_col(alpha = 0.8) +
  paletteer::scale_fill_paletteer_d("ggsci::category10_d3") +
  ggtitle("Top genres") +
  theme(plot.title = element_text(hjust = 0.5))


🤩 Det gik godt!

## 2. Mere dataudforskning.

Hvor ren er denne data? Lad os tjekke for outliers ved hjælp af boksplot. Vi vil fokusere på numeriske kolonner med færre outliers (selvom du kunne fjerne outliers). Boksplot kan vise dataens rækkevidde og hjælpe med at vælge, hvilke kolonner der skal bruges. Bemærk, at boksplot ikke viser varians, som er et vigtigt element for god data, der kan grupperes. Se venligst [denne diskussion](https://stats.stackexchange.com/questions/91536/deduce-variance-from-boxplot) for yderligere læsning.

[Boksplot](https://en.wikipedia.org/wiki/Box_plot) bruges til grafisk at afbilde fordelingen af `numerisk` data, så lad os starte med at *vælge* alle numeriske kolonner sammen med de populære musikgenrer.


In [ ]:
# Select top genre column and all other numeric columns
df_numeric <- nigerian_songs %>% 
  select(artist_top_genre, where(is.numeric)) 

# Display the data
df_numeric %>% 
  slice_head(n = 5)


Se hvordan udvælgelseshjælperen `where` gør dette nemt 💁? Udforsk andre lignende funktioner [her](https://tidyselect.r-lib.org/).

Da vi skal lave et boksplot for hver numerisk egenskab og ønsker at undgå at bruge loops, lad os omformatere vores data til et *længere* format, der giver os mulighed for at udnytte `facets` - underplots, der hver viser et delmængde af dataene.


In [ ]:
# Pivot data from wide to long
df_numeric_long <- df_numeric %>% 
  pivot_longer(!artist_top_genre, names_to = "feature_names", values_to = "values") 

# Print out data
df_numeric_long %>% 
  slice_head(n = 15)


Meget længere! Nu er det tid til nogle `ggplots`! Så hvilken `geom` vil vi bruge?


In [ ]:
# Make a box plot
df_numeric_long %>% 
  ggplot(mapping = aes(x = feature_names, y = values, fill = feature_names)) +
  geom_boxplot() +
  facet_wrap(~ feature_names, ncol = 4, scales = "free") +
  theme(legend.position = "none")


Easy-gg!

Nu kan vi se, at disse data er lidt støjende: ved at observere hver kolonne som et boxplot kan du se outliers. Du kunne gennemgå datasættet og fjerne disse outliers, men det ville gøre dataene ret minimale.

Lad os for nu vælge, hvilke kolonner vi vil bruge til vores klyngeøvelse. Lad os vælge de numeriske kolonner med lignende intervaller. Vi kunne kode `artist_top_genre` som numerisk, men vi dropper den for nu.


In [ ]:
# Select variables with similar ranges
df_numeric_select <- df_numeric %>% 
  select(popularity, danceability, acousticness, loudness, energy) 

# Normalize data
# df_numeric_select <- scale(df_numeric_select)


## 3. Beregning af k-means clustering i R

Vi kan beregne k-means i R ved hjælp af den indbyggede `kmeans`-funktion, se `help("kmeans()")`. `kmeans()`-funktionen accepterer en data frame med alle numeriske kolonner som dens primære argument.

Det første trin, når man bruger k-means clustering, er at angive antallet af klynger (k), der vil blive genereret i den endelige løsning. Vi ved, at der er 3 musikgenrer, som vi har udtrukket fra datasættet, så lad os prøve med 3:


In [ ]:
set.seed(2056)
# Kmeans clustering for 3 clusters
kclust <- kmeans(
  df_numeric_select,
  # Specify the number of clusters
  centers = 3,
  # How many random initial configurations
  nstart = 25
)

# Display clustering object
kclust


kmeans-objektet indeholder flere oplysninger, som er godt forklaret i `help("kmeans()")`. For nu vil vi fokusere på nogle få. Vi ser, at dataene er blevet grupperet i 3 klynger med størrelserne 65, 110, 111. Outputtet indeholder også klyngecentrene (gennemsnit) for de 3 grupper på tværs af de 5 variabler.

Klyngevektoren er klyngetildelingen for hver observation. Lad os bruge `augment`-funktionen til at tilføje klyngetildelingen til det oprindelige datasæt.


In [ ]:
# Add predicted cluster assignment to data set
augment(kclust, df_numeric_select) %>% 
  relocate(.cluster) %>% 
  slice_head(n = 10)


Perfekt, vi har netop opdelt vores datasæt i 3 grupper. Så, hvor god er vores klyngedannelse 🤷? Lad os tage et kig på `Silhouette score`.

### **Silhouette score**

[Silhouette-analyse](https://en.wikipedia.org/wiki/Silhouette_(clustering)) kan bruges til at undersøge afstanden mellem de resulterende klynger. Denne score varierer fra -1 til 1, og hvis scoren er tæt på 1, er klyngen tæt og godt adskilt fra andre klynger. En værdi tæt på 0 repræsenterer overlappende klynger med prøver, der ligger meget tæt på beslutningsgrænsen for de nærliggende klynger. [kilde](https://dzone.com/articles/kmeans-silhouette-score-explained-with-python-exam).

Den gennemsnitlige silhouette-metode beregner den gennemsnitlige silhouette for observationer ved forskellige værdier af *k*. En høj gennemsnitlig silhouette-score indikerer en god klyngedannelse.

Funktionen `silhouette` i cluster-pakken bruges til at beregne den gennemsnitlige silhouette-bredde.

> Silhouetten kan beregnes med enhver [distance](https://en.wikipedia.org/wiki/Distance "Distance")-metrik, såsom [Euklidisk distance](https://en.wikipedia.org/wiki/Euclidean_distance "Euclidean distance") eller [Manhattan distance](https://en.wikipedia.org/wiki/Manhattan_distance "Manhattan distance"), som vi diskuterede i [den forrige lektion](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/1-Visualize/solution/R/lesson_14-R.ipynb).


In [ ]:
# Load cluster package
library(cluster)

# Compute average silhouette score
ss <- silhouette(kclust$cluster,
                 # Compute euclidean distance
                 dist = dist(df_numeric_select))
mean(ss[, 3])


Vores score er **0,549**, altså lige midt imellem. Dette indikerer, at vores data ikke er specielt velegnede til denne type klyngedannelse. Lad os se, om vi kan bekræfte denne mistanke visuelt. [factoextra-pakken](https://rpkgs.datanovia.com/factoextra/index.html) tilbyder funktioner (`fviz_cluster()`) til at visualisere klyngedannelse.


In [ ]:
library(factoextra)

# Visualize clustering results
fviz_cluster(kclust, df_numeric_select)


Overlap i klynger indikerer, at vores data ikke er særligt velegnede til denne type klyngedannelse, men lad os fortsætte.

## 4. Bestemmelse af optimale klynger

Et grundlæggende spørgsmål, der ofte opstår i K-Means klyngedannelse, er dette - uden kendte klasselabels, hvordan ved du, hvor mange klynger du skal opdele dine data i?

En måde, vi kan forsøge at finde ud af det på, er at bruge en datasample til at `oprette en række klyngemodeller` med et stigende antal klynger (f.eks. fra 1-10) og evaluere klyngemålinger såsom **Silhouette-scoren.**

Lad os bestemme det optimale antal klynger ved at beregne klyngealgoritmen for forskellige værdier af *k* og evaluere **Within Cluster Sum of Squares** (WCSS). Den samlede within-cluster sum of square (WCSS) måler klyngens kompakthed, og vi ønsker, at den skal være så lille som muligt, hvor lavere værdier betyder, at datapunkterne er tættere på hinanden.

Lad os undersøge effekten af forskellige valg af `k`, fra 1 til 10, på denne klyngedannelse.


In [ ]:
# Create a series of clustering models
kclusts <- tibble(k = 1:10) %>% 
  # Perform kmeans clustering for 1,2,3 ... ,10 clusters
  mutate(model = map(k, ~ kmeans(df_numeric_select, centers = .x, nstart = 25)),
  # Farm out clustering metrics eg WCSS
         glanced = map(model, ~ glance(.x))) %>% 
  unnest(cols = glanced)
  

# View clustering rsulsts
kclusts


Nu hvor vi har den samlede within-cluster sum-of-squares (tot.withinss) for hver klyngedannelsesalgoritme med center *k*, bruger vi [albue-metoden](https://en.wikipedia.org/wiki/Elbow_method_(clustering)) til at finde det optimale antal klynger. Metoden består i at plotte WCSS som en funktion af antallet af klynger og vælge [kurvens albue](https://en.wikipedia.org/wiki/Elbow_of_the_curve "Albue på kurven") som det antal klynger, der skal bruges.


In [ ]:
set.seed(2056)
# Use elbow method to determine optimum number of clusters
kclusts %>% 
  ggplot(mapping = aes(x = k, y = tot.withinss)) +
  geom_line(size = 1.2, alpha = 0.8, color = "#FF7F0EFF") +
  geom_point(size = 2, color = "#FF7F0EFF")


Plottet viser en markant reduktion i WCSS (altså større *tæthed*), når antallet af klynger stiger fra én til to, og en yderligere mærkbar reduktion fra to til tre klynger. Herefter bliver reduktionen mindre markant, hvilket resulterer i en `knækpunkt` 💪 i diagrammet omkring tre klynger. Dette er en god indikation på, at der er to til tre rimeligt veladskilte klynger af datapunkter.

Vi kan nu gå videre og udtrække klynge-modellen, hvor `k = 3`:

> `pull()`: bruges til at udtrække en enkelt kolonne
>
> `pluck()`: bruges til at indeksere datastrukturer som lister


In [ ]:
# Extract k = 3 clustering
final_kmeans <- kclusts %>% 
  filter(k == 3) %>% 
  pull(model) %>% 
  pluck(1)


final_kmeans


Fantastisk! Lad os gå videre og visualisere de opnåede klynger. Har du lyst til lidt interaktivitet med `plotly`?


In [ ]:
# Add predicted cluster assignment to data set
results <-  augment(final_kmeans, df_numeric_select) %>% 
  bind_cols(df_numeric %>% select(artist_top_genre)) 

# Plot cluster assignments
clust_plt <- results %>% 
  ggplot(mapping = aes(x = popularity, y = danceability, color = .cluster, shape = artist_top_genre)) +
  geom_point(size = 2, alpha = 0.8) +
  paletteer::scale_color_paletteer_d("ggthemes::Tableau_10")

ggplotly(clust_plt)


Måske ville vi have forventet, at hver klynge (repræsenteret ved forskellige farver) ville have tydelige genrer (repræsenteret ved forskellige former).

Lad os tage et kig på modellens nøjagtighed.


In [ ]:
# Assign genres to predefined integers
label_count <- results %>% 
  group_by(artist_top_genre) %>% 
  mutate(id = cur_group_id()) %>% 
  ungroup() %>% 
  summarise(correct_labels = sum(.cluster == id))


# Print results  
cat("Result:", label_count$correct_labels, "out of", nrow(results), "samples were correctly labeled.")

cat("\nAccuracy score:", label_count$correct_labels/nrow(results))


Denne models nøjagtighed er ikke dårlig, men heller ikke fantastisk. Det kan skyldes, at dataene ikke egner sig særlig godt til K-Means Clustering. Disse data er for ubalancerede, har for lidt korrelation, og der er for stor variation mellem kolonneværdierne til at danne gode klynger. Faktisk er de klynger, der dannes, sandsynligvis stærkt påvirket eller skævvredet af de tre genrekategorier, vi definerede ovenfor.

Ikke desto mindre var det en lærerig proces!

I Scikit-learns dokumentation kan du se, at en model som denne, hvor klyngerne ikke er særlig veldefinerede, har et 'varians'-problem:

<p >
   <img src="../../images/problems.png"
   width="500"/>
   <figcaption>Infografik fra Scikit-learn</figcaption>



## **Varians**

Varians defineres som "gennemsnittet af de kvadrerede afvigelser fra gennemsnittet" [kilde](https://www.mathsisfun.com/data/standard-deviation.html). I konteksten af dette klyngeproblem refererer det til data, hvor værdierne i vores datasæt har en tendens til at afvige lidt for meget fra gennemsnittet.

✅ Dette er et godt tidspunkt til at overveje alle de måder, du kunne løse dette problem på. Justere dataene lidt mere? Bruge andre kolonner? Bruge en anden algoritme? Tip: Prøv at [skalere dine data](https://www.mygreatlearning.com/blog/learning-data-science-with-k-means-clustering/) for at normalisere dem og teste andre kolonner.

> Prøv denne '[variansberegner](https://www.calculatorsoup.com/calculators/statistics/variance-calculator.php)' for at forstå konceptet lidt bedre.

------------------------------------------------------------------------

## **🚀Udfordring**

Brug noget tid på denne notebook og juster parametrene. Kan du forbedre modellens nøjagtighed ved at rense dataene mere (for eksempel fjerne outliers)? Du kan bruge vægte til at give mere vægt til bestemte datasamples. Hvad kan du ellers gøre for at skabe bedre klynger?

Tip: Prøv at skalere dine data. Der er kommenteret kode i notebooken, der tilføjer standard skalering for at få datakolonnerne til at ligne hinanden mere i forhold til rækkevidde. Du vil opdage, at selvom silhuetscoren falder, udglattes 'knækket' i albuegrafen. Dette skyldes, at hvis dataene ikke skaleres, får data med mindre varians mere vægt. Læs lidt mere om dette problem [her](https://stats.stackexchange.com/questions/21222/are-mean-normalization-and-feature-scaling-needed-for-k-means-clustering/21226#21226).

## [**Quiz efter forelæsning**](https://gray-sand-07a10f403.1.azurestaticapps.net/quiz/30/)

## **Gennemgang & Selvstudie**

-   Tag et kig på en K-Means Simulator [som denne](https://user.ceng.metu.edu.tr/~akifakkus/courses/ceng574/k-means/). Du kan bruge dette værktøj til at visualisere eksempler på datapunkter og bestemme deres centroid. Du kan redigere dataenes tilfældighed, antal klynger og antal centroid. Hjælper dette dig med at få en idé om, hvordan dataene kan grupperes?

-   Tag også et kig på [dette handout om K-Means](https://stanford.edu/~cpiech/cs221/handouts/kmeans.html) fra Stanford.

Vil du prøve dine nyerhvervede klyngefærdigheder på datasæt, der egner sig godt til K-Means clustering? Se venligst:

-   [Træn og evaluer klyngemodeller](https://rpubs.com/eR_ic/clustering) ved hjælp af Tidymodels og lignende

-   [K-means Cluster Analysis](https://uc-r.github.io/kmeans_clustering), UC Business Analytics R Programming Guide

- [K-means clustering med tidy data-principper](https://www.tidymodels.org/learn/statistics/k-means/)

## **Opgave**

[Prøv forskellige klynge-metoder](https://github.com/microsoft/ML-For-Beginners/blob/main/5-Clustering/2-K-Means/assignment.md)

## TAK TIL:

[Jen Looper](https://www.twitter.com/jenlooper) for at skabe den originale Python-version af dette modul ♥️

[`Allison Horst`](https://twitter.com/allison_horst/) for at skabe de fantastiske illustrationer, der gør R mere imødekommende og engagerende. Find flere illustrationer i hendes [galleri](https://www.google.com/url?q=https://github.com/allisonhorst/stats-illustrations&sa=D&source=editors&ust=1626380772530000&usg=AOvVaw3zcfyCizFQZpkSLzxiiQEM).

God læring,

[Eric](https://twitter.com/ericntay), Gold Microsoft Learn Student Ambassador.

<p >
   <img src="../../images/r_learners_sm.jpeg"
   width="500"/>
   <figcaption>Kunstværk af @allison_horst</figcaption>



---

**Ansvarsfraskrivelse**:  
Dette dokument er blevet oversat ved hjælp af AI-oversættelsestjenesten [Co-op Translator](https://github.com/Azure/co-op-translator). Selvom vi bestræber os på nøjagtighed, skal du være opmærksom på, at automatiserede oversættelser kan indeholde fejl eller unøjagtigheder. Det originale dokument på dets oprindelige sprog bør betragtes som den autoritative kilde. For kritisk information anbefales professionel menneskelig oversættelse. Vi påtager os intet ansvar for misforståelser eller fejltolkninger, der måtte opstå som følge af brugen af denne oversættelse.
